# **Simple Graph Creation and GNN Model Testing**


# Work Environment


#### File Prerequisites

Run the notebook preferably on google colab.


Upload the following csvs in the path: '/thesis/Data_Preprocessing/':
* ADNI_dataset_ROIs_and_SNPs.csv
* ROIs.csv


#### Setting Google Colab as Filesystem

In [1]:
from google.colab import drive
path = "/gdrive/My Drive/thesis/Data_Preprocessing/"
drive.mount('/gdrive')
import os

Mounted at /gdrive


#### Libraries and Installations

In [2]:
import copy
import random
import time

import numpy as np
import pandas as pd

import itertools

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import trange, tqdm

In [3]:
import scipy.sparse as sp

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import networkx as nx

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

In [4]:
def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 77.2 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 45.9 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.3 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 932.1/932.1 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.0 MB/s eta 0:00:00


#### Versions

Put the following in a txt file and run the following command to instantly download the version requirements for py projects:  
pip install -r /path/to/requirements.txt

In [5]:
!python --version # python v
print("pandas ", pd.__version__) # pandas v
print("numpy ", np.__version__) # numpy v
print("torch ", torch.__version__) # torch v
# print("torchvision ", torchvision.__version__) # torchvision v
# print("pytorch lightning ", pl.__version__) # pytorch lightning v
# print("torch geometric ", pyg.__version__) # torch geometric v

Python 3.10.12
pandas  1.5.3
numpy  1.25.2
torch  2.1.0+cu121


In [6]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bi

#### Setting Up Device

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Seeds

In [8]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Datasets

#### ADNI

In [9]:
data = pd.read_csv(path + 'ADNI_dataset_ROIs_and_SNPs.csv', encoding='ISO-8859-1')

# Data Preparation


#### Studying the Data




In [10]:
pd.set_option('display.max_columns', None)
data.head()

,PTID,Age,Date,Sex,DLICV_baseline,diagnosis,MUSE_Volume_4,MUSE_Volume_11,MUSE_Volume_23,MUSE_Volume_30,MUSE_Volume_31,MUSE_Volume_32,MUSE_Volume_35,MUSE_Volume_36,MUSE_Volume_37,MUSE_Volume_38,MUSE_Volume_39,MUSE_Volume_40,MUSE_Volume_41,MUSE_Volume_47,MUSE_Volume_48,MUSE_Volume_49,MUSE_Volume_50,MUSE_Volume_51,MUSE_Volume_52,MUSE_Volume_55,MUSE_Volume_56,MUSE_Volume_57,MUSE_Volume_58,MUSE_Volume_59,MUSE_Volume_60,MUSE_Volume_61,MUSE_Volume_62,MUSE_Volume_71,MUSE_Volume_72,MUSE_Volume_73,MUSE_Volume_75,MUSE_Volume_76,MUSE_Volume_81,MUSE_Volume_82,MUSE_Volume_83,MUSE_Volume_84,MUSE_Volume_85,MUSE_Volume_86,MUSE_Volume_87,MUSE_Volume_88,MUSE_Volume_89,MUSE_Volume_90,MUSE_Volume_91,MUSE_Volume_92,MUSE_Volume_93,MUSE_Volume_94,MUSE_Volume_95,MUSE_Volume_100,MUSE_Volume_101,MUSE_Volume_102,MUSE_Volume_103,MUSE_Volume_104,MUSE_Volume_105,MUSE_Volume_106,MUSE_Volume_107,MUSE_Volume_108,MUSE_Volume_109,MUSE_Volume_112,MUSE_Volume_113,MUSE_Volume_114,MUSE_Volume_115,MUSE_Volume_116,MUSE_Volume_117,MUSE_Volume_118,MUSE_Volume_119,MUSE_Volume_120,MUSE_Volume_121,MUSE_Volume_122,MUSE_Volume_123,MUSE_Volume_124,MUSE_Volume_125,MUSE_Volume_128,MUSE_Volume_129,MUSE_Volume_132,MUSE_Volume_133,MUSE_Volume_134,MUSE_Volume_135,MUSE_Volume_136,MUSE_Volume_137,MUSE_Volume_138,MUSE_Volume_139,MUSE_Volume_140,MUSE_Volume_141,MUSE_Volume_142,MUSE_Volume_143,MUSE_Volume_144,MUSE_Volume_145,MUSE_Volume_146,MUSE_Volume_147,MUSE_Volume_148,MUSE_Volume_149,MUSE_Volume_150,MUSE_Volume_151,MUSE_Volume_152,MUSE_Volume_153,MUSE_Volume_154,MUSE_Volume_155,MUSE_Volume_156,MUSE_Volume_157,MUSE_Volume_160,MUSE_Volume_161,MUSE_Volume_162,MUSE_Volume_163,MUSE_Volume_164,MUSE_Volume_165,MUSE_Volume_166,MUSE_Volume_167,MUSE_Volume_168,MUSE_Volume_169,MUSE_Volume_170,MUSE_Volume_171,MUSE_Volume_172,MUSE_Volume_173,MUSE_Volume_174,MUSE_Volume_175,MUSE_Volume_176,MUSE_Volume_177,MUSE_Volume_178,MUSE_Volume_179,MUSE_Volume_180,MUSE_Volume_181,MUSE_Volume_182,MUSE_Volume_183,MUSE_Volume_184,MUSE_Volume_185,MUSE_Volume_186,MUSE_Volume_187,MUSE_Volume_190,MUSE_Volume_191,MUSE_Volume_192,MUSE_Volume_193,MUSE_Volume_194,MUSE_Volume_195,MUSE_Volume_196,MUSE_Volume_197,MUSE_Volume_198,MUSE_Volume_199,MUSE_Volume_200,MUSE_Volume_201,MUSE_Volume_202,MUSE_Volume_203,MUSE_Volume_204,MUSE_Volume_205,MUSE_Volume_206,MUSE_Volume_207,rs4575098,rs6656401,rs2093760,rs4844610,rs4663105,rs6733839,rs10933431,rs35349669,rs6448453,rs190982,rs9271058,rs9473117,rs9381563,rs10948363,rs2718058,rs4723711,rs1859788,rs1476679,rs12539172,rs10808026,rs7810606,rs11771145,rs28834970,rs73223431,rs4236673,rs9331896,rs11257238,rs7920721,rs3740688,rs10838725,rs983392,rs7933202,rs2081545,rs867611,rs10792832,rs3851179,rs17125924,rs17125944,rs10498633,rs12881735,rs12590654,rs442495,rs59735493,rs113260531,rs28394864,rs111278892,rs3752246,rs4147929,rs41289512,rs3865444,rs6024870,rs6014724,rs7274581,rs429358
0,002_S_0295,84.742466,2006-04-18,M,1485405.375,CN,1873.124153,1586.249283,302.695176,352.265466,1062.069832,1159.101038,20657.100036,3254.764153,3118.709527,52564.546547,52086.773326,14018.899912,14294.173225,3600.701497,3368.670352,750.937160,587.460672,16514.289409,23626.044006,1544.061802,1339.452519,4182.888734,4105.896581,7365.934170,8007.183880,4747.146291,4789.333772,4638.513528,2017.616275,2812.850291,352.265466,380.742015,107813.271569,102646.359842,26682.526999,30497.329962,51500.367341,52266.070120,63530.127527,63595.518123,426.093557,543.163817,3778.943604,3226.287604,2316.092703,2299.217710,11240.854293,4464.490169,4446.560490,3624.959299,3474.139054,1558.827420,741.444977,9202.144277,10142.925102,3385.545344,3549.021833,2759.061253,2699.998779,5983.239482,4772.458780,2428.944214,2103.045924,954.491756,976.640183,2716.873772,2089.334993,8379.488399,7090.660857,1878.397588,1371.093130,6936.676551,6667.731360,11534.057285,11793.510293,9326.597346,7320.582628,2008.124092,1507.147756,3968.787268,4167.068429,1769.764825,1258.241619,14127.532675,13099.212828,4775.622841,5759.645833,3586.990566,3536.36

In [11]:
data.describe()

,Age,DLICV_baseline,MUSE_Volume_4,MUSE_Volume_11,MUSE_Volume_23,MUSE_Volume_30,MUSE_Volume_31,MUSE_Volume_32,MUSE_Volume_35,MUSE_Volume_36,MUSE_Volume_37,MUSE_Volume_38,MUSE_Volume_39,MUSE_Volume_40,MUSE_Volume_41,MUSE_Volume_47,MUSE_Volume_48,MUSE_Volume_49,MUSE_Volume_50,MUSE_Volume_51,MUSE_Volume_52,MUSE_Volume_55,MUSE_Volume_56,MUSE_Volume_57,MUSE_Volume_58,MUSE_Volume_59,MUSE_Volume_60,MUSE_Volume_61,MUSE_Volume_62,MUSE_Volume_71,MUSE_Volume_72,MUSE_Volume_73,MUSE_Volume_75,MUSE_Volume_76,MUSE_Volume_81,MUSE_Volume_82,MUSE_Volume_83,MUSE_Volume_84,MUSE_Volume_85,MUSE_Volume_86,MUSE_Volume_87,MUSE_Volume_88,MUSE_Volume_89,MUSE_Volume_90,MUSE_Volume_91,MUSE_Volume_92,MUSE_Volume_93,MUSE_Volume_94,MUSE_Volume_95,MUSE_Volume_100,MUSE_Volume_101,MUSE_Volume_102,MUSE_Volume_103,MUSE_Volume_104,MUSE_Volume_105,MUSE_Volume_106,MUSE_Volume_107,MUSE_Volume_108,MUSE_Volume_109,MUSE_Volume_112,MUSE_Volume_113,MUSE_Volume_114,MUSE_Volume_115,MUSE_Volume_116,MUSE_Volume_117,MUSE_Volume_118,MUSE_Volume_119,MUSE_Volume_120,MUSE_Volume_121,MUSE_Volume_122,MUSE_Volume_123,MUSE_Volume_124,MUSE_Volume_125,MUSE_Volume_128,MUSE_Volume_129,MUSE_Volume_132,MUSE_Volume_133,MUSE_Volume_134,MUSE_Volume_135,MUSE_Volume_136,MUSE_Volume_137,MUSE_Volume_138,MUSE_Volume_139,MUSE_Volume_140,MUSE_Volume_141,MUSE_Volume_142,MUSE_Volume_143,MUSE_Volume_144,MUSE_Volume_145,MUSE_Volume_146,MUSE_Volume_147,MUSE_Volume_148,MUSE_Volume_149,MUSE_Volume_150,MUSE_Volume_151,MUSE_Volume_152,MUSE_Volume_153,MUSE_Volume_154,MUSE_Volume_155,MUSE_Volume_156,MUSE_Volume_157,MUSE_Volume_160,MUSE_Volume_161,MUSE_Volume_162,MUSE_Volume_163,MUSE_Volume_164,MUSE_Volume_165,MUSE_Volume_166,MUSE_Volume_167,MUSE_Volume_168,MUSE_Volume_169,MUSE_Volume_170,MUSE_Volume_171,MUSE_Volume_172,MUSE_Volume_173,MUSE_Volume_174,MUSE_Volume_175,MUSE_Volume_176,MUSE_Volume_177,MUSE_Volume_178,MUSE_Volume_179,MUSE_Volume_180,MUSE_Volume_181,MUSE_Volume_182,MUSE_Volume_183,MUSE_Volume_184,MUSE_Volume_185,MUSE_Volume_186,MUSE_Volume_187,MUSE_Volume_190,MUSE_Volume_191,MUSE_Volume_192,MUSE_Volume_193,MUSE_Volume_194,MUSE_Volume_195,MUSE_Volume_196,MUSE_Volume_197,MUSE_Volume_198,MUSE_Volume_199,MUSE_Volume_200,MUSE_Volume_201,MUSE_Volume_202,MUSE_Volume_203,MUSE_Volume_204,MUSE_Volume_205,MUSE_Volume_206,MUSE_Volume_207,rs4575098,rs6656401,rs2093760,rs4844610,rs4663105,rs6733839,rs10933431,rs35349669,rs6448453,rs190982,rs9271058,rs9473117,rs9381563,rs10948363,rs2718058,rs4723711,rs1859788,rs1476679,rs12539172,rs10808026,rs7810606,rs11771145,rs28834970,rs73223431,rs4236673,rs9331896,rs11257238,rs7920721,rs3740688,rs10838725,rs983392,rs7933202,rs2081545,rs867611,rs10792832,rs3851179,rs17125924,rs17125944,rs10498633,rs12881735,rs12590654,rs442495,rs59735493,rs113260531,rs28394864,rs111278892,rs3752246,rs4147929,rs41289512,rs3865444,rs6024870,rs6014724,rs7274581,rs429358
count,1567.000000,1.567000e+03,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,1567.000000,156

In [12]:
categorical_features = [column_name for column_name in data.columns if data[column_name].dtype == 'O']
print("Number of Categorical Features: {}".format(len(categorical_features)))
print("Categorical Features: ",categorical_features)

Number of Categorical Features: 4
Categorical Features:  ['PTID', 'Date', 'Sex', 'diagnosis']


In [13]:
numerical_features = [column_name for column_name in data.columns if data[column_name].dtype != 'O']
print("Number of Numerical Features: {}".format(len(numerical_features)))
print("Numerical Features: ",numerical_features)

Number of Numerical Features: 201
Numerical Features:  ['Age', 'DLICV_baseline', 'MUSE_Volume_4', 'MUSE_Volume_11', 'MUSE_Volume_23', 'MUSE_Volume_30', 'MUSE_Volume_31', 'MUSE_Volume_32', 'MUSE_Volume_35', 'MUSE_Volume_36', 'MUSE_Volume_37', 'MUSE_Volume_38', 'MUSE_Volume_39', 'MUSE_Volume_40', 'MUSE_Volume_41', 'MUSE_Volume_47', 'MUSE_Volume_48', 'MUSE_Volume_49', 'MUSE_Volume_50', 'MUSE_Volume_51', 'MUSE_Volume_52', 'MUSE_Volume_55', 'MUSE_Volume_56', 'MUSE_Volume_57', 'MUSE_Volume_58', 'MUSE_Volume_59', 'MUSE_Volume_60', 'MUSE_Volume_61', 'MUSE_Volume_62', 'MUSE_Volume_71', 'MUSE_Volume_72', 'MUSE_Volume_73', 'MUSE_Volume_75', 'MUSE_Volume_76', 'MUSE_Volume_81', 'MUSE_Volume_82', 'MUSE_Volume_83', 'MUSE_Volume_84', 'MUSE_Volume_85', 'MUSE_Volume_86', 'MUSE_Volume_87', 'MUSE_Volume_88', 'MUSE_Volume_89', 'MUSE_Volume_90', 'MUSE_Volume_91', 'MUSE_Volume_92', 'MUSE_Volume_93', 'MUSE_Volume_94', 'MUSE_Volume_95', 'MUSE_Volume_100', 'MUSE_Volume_101', 'MUSE_Volume_102', 'MUSE_Volume_103'

Turning categorical data to numerical.

In [14]:
data['diagnosis'] = data['diagnosis'].map({'CN': 0, 'MCI': 1, 'Dementia': 2}) #multi-class or binary?

In [15]:
data['Sex'] = data['Sex'].map({'M': 0, 'F': 1})

In [16]:
data_new= data.drop(['Date'], axis=1)
data=data_new

#### Data Preparation

In [17]:
data_controls_age = data.loc[(data['diagnosis'] == 0)]
data_controls_age_60 = data_controls_age.loc[(data_controls_age['Age'] > 60)]
data_controls = data_controls_age_60.loc[(data_controls_age_60['Age'] < 86)]
print(data_controls.shape) #controls ages 60-86

data_age_60 = data.loc[(data['Age'] > 60)]
data_age_filtered = data_age_60.loc[(data_age_60['Age'] < 86)]
print(data_age_filtered.shape) #ages 60-86

(449, 204)
(1463, 204)


In [18]:
roi_features = numerical_features[2:147]
snip_features = numerical_features[147:]
first_features = ['PTID','Sex','Age','DLICV_baseline','diagnosis']
all_but_roi_features = ['PTID','Sex','Age','DLICV_baseline','diagnosis']
all_but_roi_features.extend(snip_features)

In [19]:
print(all_but_roi_features)

['PTID', 'Sex', 'Age', 'DLICV_baseline', 'diagnosis', 'rs4575098', 'rs6656401', 'rs2093760', 'rs4844610', 'rs4663105', 'rs6733839', 'rs10933431', 'rs35349669', 'rs6448453', 'rs190982', 'rs9271058', 'rs9473117', 'rs9381563', 'rs10948363', 'rs2718058', 'rs4723711', 'rs1859788', 'rs1476679', 'rs12539172', 'rs10808026', 'rs7810606', 'rs11771145', 'rs28834970', 'rs73223431', 'rs4236673', 'rs9331896', 'rs11257238', 'rs7920721', 'rs3740688', 'rs10838725', 'rs983392', 'rs7933202', 'rs2081545', 'rs867611', 'rs10792832', 'rs3851179', 'rs17125924', 'rs17125944', 'rs10498633', 'rs12881735', 'rs12590654', 'rs442495', 'rs59735493', 'rs113260531', 'rs28394864', 'rs111278892', 'rs3752246', 'rs4147929', 'rs41289512', 'rs3865444', 'rs6024870', 'rs6014724', 'rs7274581', 'rs429358']


In [20]:
roi_mapping = pd.read_csv(path + 'ROIs.csv')

In [21]:
roi = roi_mapping.values.tolist()
roi_names = []
for i in roi:
  roi_names.append(i[0])

In [22]:
roi_names.remove('42,Right Cerebral Exterior')
roi_names.remove('43,Left Cerebral Exterior')
roi_names.remove('44,Cerebral')
roi_names.remove('45,Cerebral')
roi_names.remove('46,CSF')
roi_names.remove('63,Right vessel')
roi_names.remove('64,Left vessel')
roi_names.remove('69,Optic Chiasm')

In [23]:
ch = ','
roi_name=[]
# Remove all characters before the character ',' from string
for roi in roi_names:
  listOfWords = roi.split(ch, 1)
  if len(listOfWords) > 0:
      strValue = listOfWords[1]
      roi_name.append(strValue)

In [24]:
import warnings
warnings.filterwarnings('ignore')

# Data preprocessing & Model

Undersampling

In [37]:
# Count the number of instances in each class
class_counts = data_age_filtered['diagnosis'].value_counts()
print("Original class counts:\n", class_counts)

# Find the count of the least represented class
min_class_count = class_counts.min()

# Perform undersampling
data_balanced = pd.concat([
    data_age_filtered[data_age_filtered['diagnosis'] == class_label].sample(min_class_count, random_state=42)
    for class_label in class_counts.index
])

# Shuffle the dataset (optional but recommended)
data_balanced = data_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

print("Balanced class counts:\n", data_balanced['diagnosis'].value_counts())

Original class counts:
 1    740
0    449
2    274
Name: diagnosis, dtype: int64
Balanced class counts:
 2    274
1    274
0    274
Name: diagnosis, dtype: int64


In [38]:
data_age_filtered

,PTID,Age,Sex,DLICV_baseline,diagnosis,MUSE_Volume_4,MUSE_Volume_11,MUSE_Volume_23,MUSE_Volume_30,MUSE_Volume_31,MUSE_Volume_32,MUSE_Volume_35,MUSE_Volume_36,MUSE_Volume_37,MUSE_Volume_38,MUSE_Volume_39,MUSE_Volume_40,MUSE_Volume_41,MUSE_Volume_47,MUSE_Volume_48,MUSE_Volume_49,MUSE_Volume_50,MUSE_Volume_51,MUSE_Volume_52,MUSE_Volume_55,MUSE_Volume_56,MUSE_Volume_57,MUSE_Volume_58,MUSE_Volume_59,MUSE_Volume_60,MUSE_Volume_61,MUSE_Volume_62,MUSE_Volume_71,MUSE_Volume_72,MUSE_Volume_73,MUSE_Volume_75,MUSE_Volume_76,MUSE_Volume_81,MUSE_Volume_82,MUSE_Volume_83,MUSE_Volume_84,MUSE_Volume_85,MUSE_Volume_86,MUSE_Volume_87,MUSE_Volume_88,MUSE_Volume_89,MUSE_Volume_90,MUSE_Volume_91,MUSE_Volume_92,MUSE_Volume_93,MUSE_Volume_94,MUSE_Volume_95,MUSE_Volume_100,MUSE_Volume_101,MUSE_Volume_102,MUSE_Volume_103,MUSE_Volume_104,MUSE_Volume_105,MUSE_Volume_106,MUSE_Volume_107,MUSE_Volume_108,MUSE_Volume_109,MUSE_Volume_112,MUSE_Volume_113,MUSE_Volume_114,MUSE_Volume_115,MUSE_Volume_116,MUSE_Volume_117,MUSE_Volume_118,MUSE_Volume_119,MUSE_Volume_120,MUSE_Volume_121,MUSE_Volume_122,MUSE_Volume_123,MUSE_Volume_124,MUSE_Volume_125,MUSE_Volume_128,MUSE_Volume_129,MUSE_Volume_132,MUSE_Volume_133,MUSE_Volume_134,MUSE_Volume_135,MUSE_Volume_136,MUSE_Volume_137,MUSE_Volume_138,MUSE_Volume_139,MUSE_Volume_140,MUSE_Volume_141,MUSE_Volume_142,MUSE_Volume_143,MUSE_Volume_144,MUSE_Volume_145,MUSE_Volume_146,MUSE_Volume_147,MUSE_Volume_148,MUSE_Volume_149,MUSE_Volume_150,MUSE_Volume_151,MUSE_Volume_152,MUSE_Volume_153,MUSE_Volume_154,MUSE_Volume_155,MUSE_Volume_156,MUSE_Volume_157,MUSE_Volume_160,MUSE_Volume_161,MUSE_Volume_162,MUSE_Volume_163,MUSE_Volume_164,MUSE_Volume_165,MUSE_Volume_166,MUSE_Volume_167,MUSE_Volume_168,MUSE_Volume_169,MUSE_Volume_170,MUSE_Volume_171,MUSE_Volume_172,MUSE_Volume_173,MUSE_Volume_174,MUSE_Volume_175,MUSE_Volume_176,MUSE_Volume_177,MUSE_Volume_178,MUSE_Volume_179,MUSE_Volume_180,MUSE_Volume_181,MUSE_Volume_182,MUSE_Volume_183,MUSE_Volume_184,MUSE_Volume_185,MUSE_Volume_186,MUSE_Volume_187,MUSE_Volume_190,MUSE_Volume_191,MUSE_Volume_192,MUSE_Volume_193,MUSE_Volume_194,MUSE_Volume_195,MUSE_Volume_196,MUSE_Volume_197,MUSE_Volume_198,MUSE_Volume_199,MUSE_Volume_200,MUSE_Volume_201,MUSE_Volume_202,MUSE_Volume_203,MUSE_Volume_204,MUSE_Volume_205,MUSE_Volume_206,MUSE_Volume_207,rs4575098,rs6656401,rs2093760,rs4844610,rs4663105,rs6733839,rs10933431,rs35349669,rs6448453,rs190982,rs9271058,rs9473117,rs9381563,rs10948363,rs2718058,rs4723711,rs1859788,rs1476679,rs12539172,rs10808026,rs7810606,rs11771145,rs28834970,rs73223431,rs4236673,rs9331896,rs11257238,rs7920721,rs3740688,rs10838725,rs983392,rs7933202,rs2081545,rs867611,rs10792832,rs3851179,rs17125924,rs17125944,rs10498633,rs12881735,rs12590654,rs442495,rs59735493,rs113260531,rs28394864,rs111278892,rs3752246,rs4147929,rs41289512,rs3865444,rs6024870,rs6014724,rs7274581,rs429358
0,002_S_0295,84.742466,0,1485405.375,0,1873.124153,1586.249283,302.695176,352.265466,1062.069832,1159.101038,20657.100036,3254.764153,3118.709527,52564.546547,52086.773326,14018.899912,14294.173225,3600.701497,3368.670352,750.937160,587.460672,16514.289409,23626.044006,1544.061802,1339.452519,4182.888734,4105.896581,7365.934170,8007.183880,4747.146291,4789.333772,4638.513528,2017.616275,2812.850291,352.265466,380.742015,107813.271569,102646.359842,26682.526999,30497.329962,51500.367341,52266.070120,63530.127527,63595.518123,426.093557,543.163817,3778.943604,3226.287604,2316.092703,2299.217710,11240.854293,4464.490169,4446.560490,3624.959299,3474.139054,1558.827420,741.444977,9202.144277,10142.925102,3385.545344,3549.021833,2759.061253,2699.998779,5983.239482,4772.458780,2428.944214,2103.045924,954.491756,976.640183,2716.873772,2089.334993,8379.488399,7090.660857,1878.397588,1371.093130,6936.676551,6667.731360,11534.057285,11793.510293,9326.597346,7320.582628,2008.124092,1507.147756,3968.787268,4167.068429,1769.764825,1258.241619,14127.532675,13099.212828,4775.622841,5759.645833,3586.990566,3536.365589,942.890199,7

10-fold cross validation

In [39]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier


# Splitting dataset into train and test
X = data_age_filtered.drop(['diagnosis', 'PTID'], axis=1)
y = data_age_filtered['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Define the cross-validator
# Define the cross-validator
kf = KFold(n_splits=10, shuffle=True, random_state=42)


# Define the parameter grid for manual hyperparameter tuning
# param_grid = {
#     'n_neighbors': [3, 5, 7, 10, 15],  # Example: trying different numbers of neighbors
#     'weights': ['uniform', 'distance'],  # Example: uniform weights or distance-based
#     'metric': ['euclidean', 'manhattan']  # Example: different distance metrics
# }
# Define the KNN classifier
knn_model = KNeighborsClassifier(n_neighbors=30)

i=0

# Define a function for visualization
def visualize_data(X, y, title):
    # Separate the data based on diagnosis
    data_age_cn = X[y == 0]['Age']
    data_vol_cn = X[y == 0]['MUSE_Volume_48']

    data_age_mci = X[y == 1]['Age']
    data_vol_mci = X[y == 1]['MUSE_Volume_48']

    data_age_dem = X[y == 2]['Age']
    data_vol_dem = X[y == 2]['MUSE_Volume_48']

    # Scatter plot
    plt.scatter(data_age_cn, data_vol_cn, s=10, c='blue')
    plt.scatter(data_age_mci, data_vol_mci, s=10, c='green')
    plt.scatter(data_age_dem, data_vol_dem, s=10, c='red')

    plt.xlabel("Age (years)")
    plt.ylabel("Volume (mm^3)")
    plt.legend(["CN", "MCI", "DEM"])
    plt.title(title)
    plt.show()

# Iterate over each fold
for train_index, val_index in kf.split(X_train):
    # Split the data into the current fold's training and validation partitions
    fold_X_train, fold_X_val = X_train.iloc[train_index], X_train.iloc[val_index]
    fold_y_train, fold_y_val = y_train.iloc[train_index], y_train.iloc[val_index]

    # Visualize initial data
    # visualize_data(fold_X_train, fold_y_train, "Initial Data - Fold")

    # Linear correction
    data_controls_train = fold_X_train[fold_y_train == 0]  # Control group for the fold
    for roi_feature in roi_features:
        if roi_feature in fold_X_train.columns:
            regr = LinearRegression()
            regr.fit(data_controls_train[['Sex', 'Age', 'DLICV_baseline']], data_controls_train[roi_feature])
            # Apply correction to the training set
            correction_train = regr.predict(fold_X_train[['Sex', 'Age', 'DLICV_baseline']])
            fold_X_train[roi_feature] -= correction_train

            # Apply the same correction to the validation set
            correction_val = regr.predict(fold_X_val[['Sex', 'Age', 'DLICV_baseline']])
            fold_X_val[roi_feature] -= correction_val
    # visualize_data(fold_X_train, fold_y_train, "Linearly Corrected Data - Fold")

    # Z-normalization using control group in fold_X_train
    scaler = StandardScaler().fit(data_controls_train[roi_features])
    fold_X_train[roi_features] = scaler.transform(fold_X_train[roi_features])
    fold_X_val[roi_features] = scaler.transform(fold_X_val[roi_features])
    # visualize_data(fold_X_train, fold_y_train, "Z-Normalized Data - Fold")

    fold_X_train = fold_X_train.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)
    fold_X_val = fold_X_val.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)

    # Train the KNN model
    knn_model.fit(fold_X_train, fold_y_train)

    # Make predictions on the validation set
    y_pred_val = knn_model.predict(fold_X_val)

    # Calculate metrics (weighted because of dataset's imbalance)
    accuracy = knn_model.score(fold_X_val, fold_y_val)
    precision = precision_score(fold_y_val, y_pred_val, average='weighted')
    recall = recall_score(fold_y_val, y_pred_val, average='weighted')
    f1 = f1_score(fold_y_val, y_pred_val, average='weighted')
    auc = roc_auc_score(fold_y_val, knn_model.predict_proba(fold_X_val), multi_class='ovr', average='weighted')

    # Print the validation metrics
    print(f"Validation Metrics, Fold {i}: Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}, AUC: {auc}")
    i+=1

Validation Metrics, Fold 0: Accuracy: 0.48, Precision: 0.4911641276228801, Recall: 0.48, F1 Score: 0.4493849802371541, AUC: 0.6391716125157197
Validation Metrics, Fold 1: Accuracy: 0.6, Precision: 0.6170588344641695, Recall: 0.6, F1 Score: 0.5939223443223444, AUC: 0.7048422769567597
Validation Metrics, Fold 2: Accuracy: 0.616, Precision: 0.6436821428571429, Recall: 0.616, F1 Score: 0.5881713300249886, AUC: 0.7153595930611711
Validation Metrics, Fold 3: Accuracy: 0.47580645161290325, Precision: 0.48266567425220497, Recall: 0.47580645161290325, F1 Score: 0.4685674162106638, AUC: 0.6401079222011385
Validation Metrics, Fold 4: Accuracy: 0.5806451612903226, Precision: 0.6296121373475082, Recall: 0.5806451612903226, F1 Score: 0.546616981860382, AUC: 0.734309064745661
Validation Metrics, Fold 5: Accuracy: 0.4274193548387097, Precision: 0.478494623655914, Recall: 0.4274193548387097, F1 Score: 0.4198457223001402, AUC: 0.6312397990190891
Validation Metrics, Fold 6: Accuracy: 0.4838709677419355, 

Training on the whole train dataset and fitting on both the train and the test data

In [40]:
from sklearn.metrics import balanced_accuracy_score

# Define the control group from the training set
data_controls_train = X_train[y_train == 0]

# Linear Correction
for roi_feature in roi_features:
    if roi_feature in X_train.columns:
        regr = LinearRegression()
        regr.fit(data_controls_train[['Sex', 'Age', 'DLICV_baseline']], data_controls_train[roi_feature])
        # Apply correction to the training set
        correction_train = regr.predict(X_train[['Sex', 'Age', 'DLICV_baseline']])
        X_train[roi_feature] -= correction_train

        # Apply the same correction to the test set
        correction_test = regr.predict(X_test[['Sex', 'Age', 'DLICV_baseline']])
        X_test[roi_feature] -= correction_test

X_train = X_train.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)
X_test = X_test.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)

# Z-Normalization
scaler = StandardScaler().fit(data_controls_train[roi_features])
X_train[roi_features] = scaler.transform(X_train[roi_features])
X_test[roi_features] = scaler.transform(X_test[roi_features])

# After cross-validation, train the model on the entire training set
knn_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_test = knn_model.predict(X_test)

# Calculate and print standard accuracy for the test set
test_accuracy = knn_model.score(X_test, y_test)
print(f"Test set accuracy: {test_accuracy}")

# Calculate and print balanced accuracy for the test set
test_balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
print(f"Test set Balanced Accuracy: {test_balanced_accuracy}")

# Calculate and print metrics for the test set
test_precision = precision_score(y_test, y_pred_test, average='weighted')
test_recall = recall_score(y_test, y_pred_test, average='weighted')
test_f1 = f1_score(y_test, y_pred_test, average='weighted')
test_auc = roc_auc_score(y_test, knn_model.predict_proba(X_test), multi_class='ovr', average='weighted')

print(f"Test set Precision: {test_precision}, Recall: {test_recall}, F1 Score: {test_f1}, AUC: {test_auc}")


Test set accuracy: 0.4681818181818182
Test set Balanced Accuracy: 0.4415488042071307
Test set Precision: 0.48012741361637795, Recall: 0.4681818181818182, F1 Score: 0.4427316404851951, AUC: 0.6243919510270833


In [41]:
X_train

,MUSE_Volume_4,MUSE_Volume_11,MUSE_Volume_23,MUSE_Volume_30,MUSE_Volume_31,MUSE_Volume_32,MUSE_Volume_35,MUSE_Volume_36,MUSE_Volume_37,MUSE_Volume_38,MUSE_Volume_39,MUSE_Volume_40,MUSE_Volume_41,MUSE_Volume_47,MUSE_Volume_48,MUSE_Volume_49,MUSE_Volume_50,MUSE_Volume_51,MUSE_Volume_52,MUSE_Volume_55,MUSE_Volume_56,MUSE_Volume_57,MUSE_Volume_58,MUSE_Volume_59,MUSE_Volume_60,MUSE_Volume_61,MUSE_Volume_62,MUSE_Volume_71,MUSE_Volume_72,MUSE_Volume_73,MUSE_Volume_75,MUSE_Volume_76,MUSE_Volume_81,MUSE_Volume_82,MUSE_Volume_83,MUSE_Volume_84,MUSE_Volume_85,MUSE_Volume_86,MUSE_Volume_87,MUSE_Volume_88,MUSE_Volume_89,MUSE_Volume_90,MUSE_Volume_91,MUSE_Volume_92,MUSE_Volume_93,MUSE_Volume_94,MUSE_Volume_95,MUSE_Volume_100,MUSE_Volume_101,MUSE_Volume_102,MUSE_Volume_103,MUSE_Volume_104,MUSE_Volume_105,MUSE_Volume_106,MUSE_Volume_107,MUSE_Volume_108,MUSE_Volume_109,MUSE_Volume_112,MUSE_Volume_113,MUSE_Volume_114,MUSE_Volume_115,MUSE_Volume_116,MUSE_Volume_117,MUSE_Volume_118,MUSE_Volume_119,MUSE_Volume_120,MUSE_Volume_121,MUSE_Volume_122,MUSE_Volume_123,MUSE_Volume_124,MUSE_Volume_125,MUSE_Volume_128,MUSE_Volume_129,MUSE_Volume_132,MUSE_Volume_133,MUSE_Volume_134,MUSE_Volume_135,MUSE_Volume_136,MUSE_Volume_137,MUSE_Volume_138,MUSE_Volume_139,MUSE_Volume_140,MUSE_Volume_141,MUSE_Volume_142,MUSE_Volume_143,MUSE_Volume_144,MUSE_Volume_145,MUSE_Volume_146,MUSE_Volume_147,MUSE_Volume_148,MUSE_Volume_149,MUSE_Volume_150,MUSE_Volume_151,MUSE_Volume_152,MUSE_Volume_153,MUSE_Volume_154,MUSE_Volume_155,MUSE_Volume_156,MUSE_Volume_157,MUSE_Volume_160,MUSE_Volume_161,MUSE_Volume_162,MUSE_Volume_163,MUSE_Volume_164,MUSE_Volume_165,MUSE_Volume_166,MUSE_Volume_167,MUSE_Volume_168,MUSE_Volume_169,MUSE_Volume_170,MUSE_Volume_171,MUSE_Volume_172,MUSE_Volume_173,MUSE_Volume_174,MUSE_Volume_175,MUSE_Volume_176,MUSE_Volume_177,MUSE_Volume_178,MUSE_Volume_179,MUSE_Volume_180,MUSE_Volume_181,MUSE_Volume_182,MUSE_Volume_183,MUSE_Volume_184,MUSE_Volume_185,MUSE_Volume_186,MUSE_Volume_187,MUSE_Volume_190,MUSE_Volume_191,MUSE_Volume_192,MUSE_Volume_193,MUSE_Volume_194,MUSE_Volume_195,MUSE_Volume_196,MUSE_Volume_197,MUSE_Volume_198,MUSE_Volume_199,MUSE_Volume_200,MUSE_Volume_201,MUSE_Volume_202,MUSE_Volume_203,MUSE_Volume_204,MUSE_Volume_205,MUSE_Volume_206,MUSE_Volume_207,rs4575098,rs6656401,rs2093760,rs4844610,rs4663105,rs6733839,rs10933431,rs35349669,rs6448453,rs190982,rs9271058,rs9473117,rs9381563,rs10948363,rs2718058,rs4723711,rs1859788,rs1476679,rs12539172,rs10808026,rs7810606,rs11771145,rs28834970,rs73223431,rs4236673,rs9331896,rs11257238,rs7920721,rs3740688,rs10838725,rs983392,rs7933202,rs2081545,rs867611,rs10792832,rs3851179,rs17125924,rs17125944,rs10498633,rs12881735,rs12590654,rs442495,rs59735493,rs113260531,rs28394864,rs111278892,rs3752246,rs4147929,rs41289512,rs3865444,rs6024870,rs6014724,rs7274581,rs429358
512,-1.948789,-3.519628,-6.864576,-7.094928,-8.650670,-8.982399,-8.842779,-6.923820,-8.182858,-9.977695,-10.012131,-9.037972,-9.857895,-9.708276,-10.364709,-0.142393,-0.055402,0.030433,0.302972,-9.157386,-9.346332,-8.979470,-9.637544,-12.712174,-12.822045,-9.894595,-9.703516,-6.102152,-7.726123,-6.333825,-2.633948,-1.630805,-8.063320,-8.203913,-7.216565,-7.159445,-8.777602,-8.325936,-8.389574,-8.803952,-6.036176,-7.158957,-7.537406,-7.070318,-7.335203,-7.715266,-7.462522,-6.514412,-6.046324,-7.996275,-8.673601,-4.885418,-4.193093,-6.554702,-5.105966,-5.017504,-5.733212,-6.192827,-5.549694,-8.559681,-8.020253,-7.053861,-8.356034,-6.292032,-3.896864,-5.588596,-5.247532,-6.559102,-7.080791,-6.818210,-6.048775,-5.720854,-6.408606,-7.265054,-8.219865,-8.397948,-7.023477,-4.746934,-4.704919,-7.188947,-7.803646,-5.033622,-3.270467,-6.652709,-6.591331,-4.628801,-6.724356,-6.976084,-7.474191,-4.906424,-3.031026,-4.351703,-4.244168,-4.921928,-6.569336,-8.845398,-9.714961,-3.600114,-4.549492,-5.223332,-7.693628,-4.638961,-3.904311,-2.882125,-2.760562,-5.791632,-6.679109,-7.548102,-7.874723,-7.560746,-8.456730,-6.391372,-5.388566,-4.622870,-4.559614,-7.205274,-8.

In [42]:
X_test

,MUSE_Volume_4,MUSE_Volume_11,MUSE_Volume_23,MUSE_Volume_30,MUSE_Volume_31,MUSE_Volume_32,MUSE_Volume_35,MUSE_Volume_36,MUSE_Volume_37,MUSE_Volume_38,MUSE_Volume_39,MUSE_Volume_40,MUSE_Volume_41,MUSE_Volume_47,MUSE_Volume_48,MUSE_Volume_49,MUSE_Volume_50,MUSE_Volume_51,MUSE_Volume_52,MUSE_Volume_55,MUSE_Volume_56,MUSE_Volume_57,MUSE_Volume_58,MUSE_Volume_59,MUSE_Volume_60,MUSE_Volume_61,MUSE_Volume_62,MUSE_Volume_71,MUSE_Volume_72,MUSE_Volume_73,MUSE_Volume_75,MUSE_Volume_76,MUSE_Volume_81,MUSE_Volume_82,MUSE_Volume_83,MUSE_Volume_84,MUSE_Volume_85,MUSE_Volume_86,MUSE_Volume_87,MUSE_Volume_88,MUSE_Volume_89,MUSE_Volume_90,MUSE_Volume_91,MUSE_Volume_92,MUSE_Volume_93,MUSE_Volume_94,MUSE_Volume_95,MUSE_Volume_100,MUSE_Volume_101,MUSE_Volume_102,MUSE_Volume_103,MUSE_Volume_104,MUSE_Volume_105,MUSE_Volume_106,MUSE_Volume_107,MUSE_Volume_108,MUSE_Volume_109,MUSE_Volume_112,MUSE_Volume_113,MUSE_Volume_114,MUSE_Volume_115,MUSE_Volume_116,MUSE_Volume_117,MUSE_Volume_118,MUSE_Volume_119,MUSE_Volume_120,MUSE_Volume_121,MUSE_Volume_122,MUSE_Volume_123,MUSE_Volume_124,MUSE_Volume_125,MUSE_Volume_128,MUSE_Volume_129,MUSE_Volume_132,MUSE_Volume_133,MUSE_Volume_134,MUSE_Volume_135,MUSE_Volume_136,MUSE_Volume_137,MUSE_Volume_138,MUSE_Volume_139,MUSE_Volume_140,MUSE_Volume_141,MUSE_Volume_142,MUSE_Volume_143,MUSE_Volume_144,MUSE_Volume_145,MUSE_Volume_146,MUSE_Volume_147,MUSE_Volume_148,MUSE_Volume_149,MUSE_Volume_150,MUSE_Volume_151,MUSE_Volume_152,MUSE_Volume_153,MUSE_Volume_154,MUSE_Volume_155,MUSE_Volume_156,MUSE_Volume_157,MUSE_Volume_160,MUSE_Volume_161,MUSE_Volume_162,MUSE_Volume_163,MUSE_Volume_164,MUSE_Volume_165,MUSE_Volume_166,MUSE_Volume_167,MUSE_Volume_168,MUSE_Volume_169,MUSE_Volume_170,MUSE_Volume_171,MUSE_Volume_172,MUSE_Volume_173,MUSE_Volume_174,MUSE_Volume_175,MUSE_Volume_176,MUSE_Volume_177,MUSE_Volume_178,MUSE_Volume_179,MUSE_Volume_180,MUSE_Volume_181,MUSE_Volume_182,MUSE_Volume_183,MUSE_Volume_184,MUSE_Volume_185,MUSE_Volume_186,MUSE_Volume_187,MUSE_Volume_190,MUSE_Volume_191,MUSE_Volume_192,MUSE_Volume_193,MUSE_Volume_194,MUSE_Volume_195,MUSE_Volume_196,MUSE_Volume_197,MUSE_Volume_198,MUSE_Volume_199,MUSE_Volume_200,MUSE_Volume_201,MUSE_Volume_202,MUSE_Volume_203,MUSE_Volume_204,MUSE_Volume_205,MUSE_Volume_206,MUSE_Volume_207,rs4575098,rs6656401,rs2093760,rs4844610,rs4663105,rs6733839,rs10933431,rs35349669,rs6448453,rs190982,rs9271058,rs9473117,rs9381563,rs10948363,rs2718058,rs4723711,rs1859788,rs1476679,rs12539172,rs10808026,rs7810606,rs11771145,rs28834970,rs73223431,rs4236673,rs9331896,rs11257238,rs7920721,rs3740688,rs10838725,rs983392,rs7933202,rs2081545,rs867611,rs10792832,rs3851179,rs17125924,rs17125944,rs10498633,rs12881735,rs12590654,rs442495,rs59735493,rs113260531,rs28394864,rs111278892,rs3752246,rs4147929,rs41289512,rs3865444,rs6024870,rs6014724,rs7274581,rs429358
1265,-3.309464,-4.589947,-5.964318,-6.509761,-9.947689,-10.780832,-9.899748,-7.468103,-8.306570,-9.352233,-9.396177,-9.058342,-9.188435,-10.534271,-11.049213,1.695588,0.398668,-1.916758,-2.108035,-10.899753,-11.506332,-9.845572,-9.939792,-12.738586,-12.791460,-11.685824,-11.397368,-7.009572,-6.922915,-6.419431,-3.486047,-2.997706,-8.243422,-7.530000,-6.153934,-6.000205,-7.516836,-7.653246,-8.959659,-8.470608,-7.913952,-9.257210,-9.771446,-9.640706,-9.152201,-8.965747,-5.436624,-6.759787,-6.196322,-8.482249,-8.650926,-5.414305,-4.544198,-6.212716,-5.792334,-4.578755,-5.010197,-4.544963,-5.145586,-4.837992,-5.648015,-8.742161,-9.839436,-3.687005,-4.859218,-6.182925,-5.293910,-8.853400,-8.039916,-4.181795,-4.881480,-5.606954,-4.923812,-6.836528,-8.221920,-7.567308,-7.782377,-4.833783,-3.128559,-6.227546,-6.310260,-5.355004,-4.996329,-7.178063,-7.121728,-4.822446,-5.467314,-6.797282,-7.181904,-3.148319,-3.249608,-5.192592,-5.912716,-4.636811,-6.412822,-7.626504,-7.535902,-3.838355,-3.416350,-5.167452,-6.762384,-5.734198,-3.785097,-2.769875,-2.769170,-5.870201,-6.156860,-7.336142,-7.758991,-8.930510,-9.740600,-7.139965,-6.782006,-4.115482,-3.295785,-5.1242

A few considerations should be kept in mind to further ensure that our approach is unbiased:

* Representation of Control Group: Make sure that the control group (CN) in the training data is representative of the normal population.

* Generalizability of Corrections: Applying corrections based solely on the control group assumes that the relationship between predictors (like age, sex) and the outcome (ROI features) is the same in control and diseased groups. If this assumption does not hold, the model may not capture disease-specific patterns effectively.

* Statistical Assumptions: Linear regression makes certain assumptions (like linearity, normality, homoscedasticity, and independence of errors). Ensure these assumptions hold for your data; otherwise, the corrections might be inappropriate.

Nested CV with GridSearch

In [43]:
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

X = data_age_filtered.drop(['diagnosis', 'PTID'], axis=1)
y = data_age_filtered['diagnosis']

# Assuming X, y, roi_features are defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

param_grid = {
    'n_neighbors': [3, 5, 7, 10, 15, 30],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean']
}

outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

outer_fold_results = []

for train_index, test_index in outer_cv.split(X_train):
    X_train_outer, X_val_outer = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_outer, y_val_outer = y_train.iloc[train_index], y_train.iloc[test_index]

    inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
    grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=inner_cv, scoring='accuracy', n_jobs=-1)

    for inner_train_index, inner_val_index in inner_cv.split(X_train_outer):
        X_train_inner, X_val_inner = X_train_outer.iloc[inner_train_index], X_train_outer.iloc[inner_val_index]
        y_train_inner, y_val_inner = y_train_outer.iloc[inner_train_index], y_train_outer.iloc[inner_val_index]

        # Preprocess for the inner fold
        data_controls_train_inner = X_train_inner[y_train_inner == 0]
        for roi_feature in roi_features:
            if roi_feature in X_train_inner.columns:
                regr = LinearRegression()
                regr.fit(data_controls_train_inner[['Sex', 'Age', 'DLICV_baseline']], data_controls_train_inner[roi_feature])
                # Apply correction to the training set
                correction_train = regr.predict(X_train_inner[['Sex', 'Age', 'DLICV_baseline']])
                X_train_inner[roi_feature] -= correction_train

                # Apply the same correction to the validation set
                correction_val = regr.predict(X_val_inner[['Sex', 'Age', 'DLICV_baseline']])
                X_val_inner[roi_feature] -= correction_val

        # Z-normalization for training and validation sets
        scaler = StandardScaler().fit(data_controls_train_inner[roi_features])
        X_train_inner[roi_features] = scaler.transform(X_train_inner[roi_features])
        X_val_inner[roi_features] = scaler.transform(X_val_inner[roi_features])

        X_train_inner = X_train_inner.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)
        X_val_inner = X_val_inner.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)

        grid_search.fit(X_train_inner, y_train_inner)

    # Evaluate the best model on the outer validation set
    best_model = grid_search.best_estimator_

    # We apply the linear correction and the z normalisation of the outer
    # loop after we are doe with the inner loop, in order to avoid applying
    # these filter twice.
    data_controls_train_outer = X_train_outer[y_train_outer == 0]
    for roi_feature in roi_features:
        if roi_feature in X_train_outer.columns:
            regr = LinearRegression()
            regr.fit(data_controls_train_outer[['Sex', 'Age', 'DLICV_baseline']], data_controls_train_outer[roi_feature])
            # Apply correction to the training set
            correction_train = regr.predict(X_train_outer[['Sex', 'Age', 'DLICV_baseline']])
            X_train_outer[roi_feature] -= correction_train

            # Apply the same correction to the validation set
            correction_val = regr.predict(X_val_outer[['Sex', 'Age', 'DLICV_baseline']])
            X_val_outer[roi_feature] -= correction_val

    # Z-normalization for training and validation sets
    scaler = StandardScaler().fit(data_controls_train_outer[roi_features])
    X_train_outer[roi_features] = scaler.transform(X_train_outer[roi_features])
    X_val_outer[roi_features] = scaler.transform(X_val_outer[roi_features])

    X_train_outer = X_train_outer.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)
    X_val_outer = X_val_outer.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)


    y_pred_val = best_model.predict(X_val_outer)

    # # Calculate and print metrics for each outer fold
    # accuracy = best_model.score(X_val_outer, y_val_outer)
    # print(f"Outer Fold Metrics: Accuracy: {accuracy}, Balanced Accuracy: {balanced_accuracy_score(y_val_outer, y_pred_val)}, ...")

    # Append metrics to dict
    fold_metrics = {
        "Accuracy": best_model.score(X_val_outer, y_val_outer),
        "Balanced Accuracy": balanced_accuracy_score(y_val_outer, y_pred_val),
        "Precision": precision_score(y_val_outer, y_pred_val, average='weighted'),
        "Recall": recall_score(y_val_outer, y_pred_val, average='weighted'),
        "F1 Score": f1_score(y_val_outer, y_pred_val, average='weighted'),
        "AUC": roc_auc_score(y_val_outer, best_model.predict_proba(X_val_outer), multi_class='ovr', average='weighted')
    }
    outer_fold_results.append(fold_metrics)
    print(f"Outer Fold Metrics: {fold_metrics}")

# Average metrics across all outer folds
avg_metrics = {metric: np.mean([fold[metric] for fold in outer_fold_results]) for metric in outer_fold_results[0]}
print("\nAverage Metrics Across All Outer Folds:")
print(avg_metrics)



Outer Fold Metrics: {'Accuracy': 0.5261044176706827, 'Balanced Accuracy': 0.5438342934405926, 'Precision': 0.541455697481864, 'Recall': 0.5261044176706827, 'F1 Score': 0.524197643029713, 'AUC': 0.6545805117264375}
Outer Fold Metrics: {'Accuracy': 0.5461847389558233, 'Balanced Accuracy': 0.48808061333014846, 'Precision': 0.5498400220266408, 'Recall': 0.5461847389558233, 'F1 Score': 0.5033236844813651, 'AUC': 0.6991129134026501}
Outer Fold Metrics: {'Accuracy': 0.4819277108433735, 'Balanced Accuracy': 0.50904838585998, 'Precision': 0.524770410312579, 'Recall': 0.4819277108433735, 'F1 Score': 0.46561906856635465, 'AUC': 0.6521063197965931}
Outer Fold Metrics: {'Accuracy': 0.592741935483871, 'Balanced Accuracy': 0.5621432549963464, 'Precision': 0.5922003842414023, 'Recall': 0.592741935483871, 'F1 Score': 0.5910676531188976, 'AUC': 0.7102515659010042}
Outer Fold Metrics: {'Accuracy': 0.5483870967741935, 'Balanced Accuracy': 0.505707643126998, 'Precision': 0.5435227854582693, 'Recall': 0.548

In [44]:
# Retrieve the best hyperparameters from the grid search
best_params = grid_search.best_params_

# Retrain the model on the entire training dataset (X_train, y_train) with these parameters
final_knn_model = KNeighborsClassifier(**best_params)

# Apply preprocessing (linear correction and Z-normalization) to the entire training dataset X_train
data_controls = X_train[y_train == 0]  # Control group for the training dataset
for roi_feature in roi_features:
    if roi_feature in X_train.columns:
        regr = LinearRegression()
        regr.fit(data_controls[['Sex', 'Age', 'DLICV_baseline']], data_controls[roi_feature])
        # Apply correction to the training set
        correction_train = regr.predict(X_train[['Sex', 'Age', 'DLICV_baseline']])
        X_train[roi_feature] -= correction_train

        # Apply the same correction to the test set
        correction_test = regr.predict(X_test[['Sex', 'Age', 'DLICV_baseline']])
        X_test[roi_feature] -= correction_test


# Z-normalization
scaler = StandardScaler().fit(data_controls[roi_features])
X_train[roi_features] = scaler.transform(X_train[roi_features])
X_test[roi_features] = scaler.transform(X_test[roi_features])


X_train = X_train.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)
X_test = X_test.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)

# Train the final model on the entire training dataset
final_knn_model.fit(X_train, y_train)

# # Apply linear correction and Z-normalization to X_test
# for roi_feature in roi_features:
#     if roi_feature in X_test.columns:
#         correction_test = regr.predict(X_test[['Sex', 'Age', 'DLICV_baseline']])
#         X_test[roi_feature] -= correction_test
# X_test[roi_features] = scaler.transform(X_test[roi_features])
# X_test = X_test.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)

# Evaluate the final model on the test set (X_test, y_test)
y_pred_test = final_knn_model.predict(X_test)
test_accuracy = final_knn_model.score(X_test, y_test)
test_balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test, average='weighted')
test_recall = recall_score(y_test, y_pred_test, average='weighted')
test_f1 = f1_score(y_test, y_pred_test, average='weighted')
test_auc = roc_auc_score(y_test, final_knn_model.predict_proba(X_test), multi_class='ovr', average='weighted')

print(f"Test set Metrics: Accuracy: {test_accuracy}, Balanced Accuracy: {test_balanced_accuracy}, Precision: {test_precision}, Recall: {test_recall}, F1 Score: {test_f1}, AUC: {test_auc}")

Test set Metrics: Accuracy: 0.4909090909090909, Balanced Accuracy: 0.47158876562369395, Precision: 0.5185294325239977, Recall: 0.4909090909090909, F1 Score: 0.47370538801022527, AUC: 0.6329923516691478


In [32]:
best_params

{'metric': 'euclidean', 'n_neighbors': 30, 'weights': 'uniform'}

#### Nested CV with Randomized Search

In [52]:
from sklearn.model_selection import RandomizedSearchCV, KFold, train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats

X = data_age_filtered.drop(['diagnosis', 'PTID'], axis=1)
y = data_age_filtered['diagnosis']

# Assuming X, y, roi_features are defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

# Define parameter distribution (instead of a fixed grid)
param_dist = {
    'n_neighbors': stats.randint(3, 1000),  # Uniformly distributed integers from 3 to 30
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)

outer_fold_results = []

for train_index, test_index in outer_cv.split(X_train):
    X_train_outer, X_val_outer = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_outer, y_val_outer = y_train.iloc[train_index], y_train.iloc[test_index]

    inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

    # Use RandomizedSearchCV
    n_iter_search = 20  # Number of parameter settings sampled
    random_search = RandomizedSearchCV(KNeighborsClassifier(), param_distributions=param_dist, n_iter=n_iter_search, cv=inner_cv, scoring='accuracy', n_jobs=-1, random_state=42)

    for inner_train_index, inner_val_index in inner_cv.split(X_train_outer):
        X_train_inner, X_val_inner = X_train_outer.iloc[inner_train_index], X_train_outer.iloc[inner_val_index]
        y_train_inner, y_val_inner = y_train_outer.iloc[inner_train_index], y_train_outer.iloc[inner_val_index]

        # Preprocess for the inner fold
        data_controls_train_inner = X_train_inner[y_train_inner == 0]
        for roi_feature in roi_features:
            if roi_feature in X_train_inner.columns:
                regr = LinearRegression()
                regr.fit(data_controls_train_inner[['Sex', 'Age', 'DLICV_baseline']], data_controls_train_inner[roi_feature])
                # Apply correction to the training set
                correction_train = regr.predict(X_train_inner[['Sex', 'Age', 'DLICV_baseline']])
                X_train_inner[roi_feature] -= correction_train

                # Apply the same correction to the validation set
                correction_val = regr.predict(X_val_inner[['Sex', 'Age', 'DLICV_baseline']])
                X_val_inner[roi_feature] -= correction_val

        # Z-normalization for training and validation sets
        scaler = StandardScaler().fit(data_controls_train_inner[roi_features])
        X_train_inner[roi_features] = scaler.transform(X_train_inner[roi_features])
        X_val_inner[roi_features] = scaler.transform(X_val_inner[roi_features])

        X_train_inner = X_train_inner.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)
        X_val_inner = X_val_inner.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)

        random_search.fit(X_train_inner, y_train_inner)

    # Evaluate the best model on the outer validation set
    best_model = grid_search.best_estimator_

    # We apply the linear correction and the z normalisation of the outer
    # loop after we are doe with the inner loop, in order to avoid applying
    # these filter twice.
    data_controls_train_outer = X_train_outer[y_train_outer == 0]
    for roi_feature in roi_features:
        if roi_feature in X_train_outer.columns:
            regr = LinearRegression()
            regr.fit(data_controls_train_outer[['Sex', 'Age', 'DLICV_baseline']], data_controls_train_outer[roi_feature])
            # Apply correction to the training set
            correction_train = regr.predict(X_train_outer[['Sex', 'Age', 'DLICV_baseline']])
            X_train_outer[roi_feature] -= correction_train

            # Apply the same correction to the validation set
            correction_val = regr.predict(X_val_outer[['Sex', 'Age', 'DLICV_baseline']])
            X_val_outer[roi_feature] -= correction_val

    # Z-normalization for training and validation sets
    scaler = StandardScaler().fit(data_controls_train_outer[roi_features])
    X_train_outer[roi_features] = scaler.transform(X_train_outer[roi_features])
    X_val_outer[roi_features] = scaler.transform(X_val_outer[roi_features])

    X_train_outer = X_train_outer.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)
    X_val_outer = X_val_outer.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)


    y_pred_val = best_model.predict(X_val_outer)
    # Calculate and print metrics for each outer fold
    accuracy = best_model.score(X_val_outer, y_val_outer)
    print(f"Outer Fold Metrics: Accuracy: {accuracy}, Balanced Accuracy: {balanced_accuracy_score(y_val_outer, y_pred_val)}, ...")

    # Append metrics to dict
    fold_metrics = {
        "Accuracy": best_model.score(X_val_outer, y_val_outer),
        "Balanced Accuracy": balanced_accuracy_score(y_val_outer, y_pred_val),
        "Precision": precision_score(y_val_outer, y_pred_val, average='macro'),
        "Recall": recall_score(y_val_outer, y_pred_val, average='macro'),
        "F1 Score": f1_score(y_val_outer, y_pred_val, average='macro'),
        "AUC": roc_auc_score(y_val_outer, best_model.predict_proba(X_val_outer), multi_class='ovr', average='macro')
    }

    outer_fold_results.append(fold_metrics)
    print(f"Outer Fold Metrics: {fold_metrics}")

# Average metrics across all outer folds
avg_metrics = {metric: np.mean([fold[metric] for fold in outer_fold_results]) for metric in outer_fold_results[0]}
print("\nAverage Metrics Across All Outer Folds:")
print(avg_metrics)



Outer Fold Metrics: Accuracy: 0.5742971887550201, Balanced Accuracy: 0.5233277658474509, ...
Outer Fold Metrics: {'Accuracy': 0.5742971887550201, 'Balanced Accuracy': 0.5233277658474509, 'Precision': 0.7129629629629629, 'Recall': 0.5233277658474509, 'F1 Score': 0.4886280806983572, 'AUC': 0.8455802086677028}
Outer Fold Metrics: Accuracy: 0.5542168674698795, Balanced Accuracy: 0.4881886740944224, ...
Outer Fold Metrics: {'Accuracy': 0.5542168674698795, 'Balanced Accuracy': 0.4881886740944224, 'Precision': 0.705850436423685, 'Recall': 0.4881886740944224, 'F1 Score': 0.42644111062460066, 'AUC': 0.8262809783490722}
Outer Fold Metrics: Accuracy: 0.5542168674698795, Balanced Accuracy: 0.5450885668276974, ...
Outer Fold Metrics: {'Accuracy': 0.5542168674698795, 'Balanced Accuracy': 0.5450885668276974, 'Precision': 0.6741519350215003, 'Recall': 0.5450885668276974, 'F1 Score': 0.5098015357992646, 'AUC': 0.8404921981038395}
Outer Fold Metrics: Accuracy: 0.5846774193548387, Balanced Accuracy: 0.54

In [53]:
# Retrieve the best hyperparameters from the grid search
best_params = grid_search.best_params_

# Retrain the model on the entire training dataset (X_train, y_train) with these parameters
final_knn_model = KNeighborsClassifier(**best_params)

# Apply preprocessing (linear correction and Z-normalization) to the entire training dataset X_train
data_controls = X_train[y_train == 0]  # Control group for the training dataset
for roi_feature in roi_features:
    if roi_feature in X_train.columns:
        regr = LinearRegression()
        regr.fit(data_controls[['Sex', 'Age', 'DLICV_baseline']], data_controls[roi_feature])
        # Apply correction to the training set
        correction_train = regr.predict(X_train[['Sex', 'Age', 'DLICV_baseline']])
        X_train[roi_feature] -= correction_train

        # Apply the same correction to the test set
        correction_test = regr.predict(X_test[['Sex', 'Age', 'DLICV_baseline']])
        X_test[roi_feature] -= correction_test


# Z-normalization
scaler = StandardScaler().fit(data_controls[roi_features])
X_train[roi_features] = scaler.transform(X_train[roi_features])
X_test[roi_features] = scaler.transform(X_test[roi_features])


X_train = X_train.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)
X_test = X_test.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)

# Train the final model on the entire training dataset
final_knn_model.fit(X_train, y_train)

# # Apply linear correction and Z-normalization to X_test
# for roi_feature in roi_features:
#     if roi_feature in X_test.columns:
#         correction_test = regr.predict(X_test[['Sex', 'Age', 'DLICV_baseline']])
#         X_test[roi_feature] -= correction_test
# X_test[roi_features] = scaler.transform(X_test[roi_features])
# X_test = X_test.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)

# Evaluate the final model on the test set (X_test, y_test)
y_pred_test = final_knn_model.predict(X_test)
test_accuracy = final_knn_model.score(X_test, y_test)
test_balanced_accuracy = balanced_accuracy_score(y_test, y_pred_test)
test_precision = precision_score(y_test, y_pred_test, average='weighted')
test_recall = recall_score(y_test, y_pred_test, average='weighted')
test_f1 = f1_score(y_test, y_pred_test, average='weighted')
test_auc = roc_auc_score(y_test, final_knn_model.predict_proba(X_test), multi_class='ovr', average='weighted')

print(f"Test set Metrics: Accuracy: {test_accuracy}, Balanced Accuracy: {test_balanced_accuracy}, Precision: {test_precision}, Recall: {test_recall}, F1 Score: {test_f1}, AUC: {test_auc}")

Test set Metrics: Accuracy: 0.4909090909090909, Balanced Accuracy: 0.47158876562369395, Precision: 0.5185294325239977, Recall: 0.4909090909090909, F1 Score: 0.47370538801022527, AUC: 0.6329923516691478


In [50]:
best_params

{'metric': 'euclidean', 'n_neighbors': 30, 'weights': 'distance'}

##### back ups


In [36]:
# only corrected and standardized outer folds
# (which include the inner folds, but it is the same problem of data
# leakage even though the whole data are train data. Even though all of these
# data are train data, we should treat them as validation/test data for validating
# and evaluating the model.)

# from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
# from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, balanced_accuracy_score
# from sklearn.linear_model import LinearRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.preprocessing import StandardScaler

# # Splitting dataset into train and test
# X = data_age_filtered.drop(['diagnosis', 'PTID'], axis=1)
# y = data_age_filtered['diagnosis']

# # Assuming X, y, roi_features are defined
# # Split the dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Define parameter grid for KNN
# param_grid = {
#     'n_neighbors': [3, 5, 7, 10, 15, 30],
#     'weights': ['uniform', 'distance'],
#     'metric': ['euclidean', 'manhattan']
# }

# # Define outer cross-validation
# outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# # Lists to store metrics for each outer fold
# outer_fold_accuracy = []
# outer_fold_balanced_accuracy = []
# outer_fold_precision = []
# outer_fold_recall = []
# outer_fold_f1 = []
# outer_fold_auc = []

# # Outer loop for model evaluation
# for train_index, test_index in outer_cv.split(X_train, y_train):
#     # Split training data into training and validation for the current outer fold
#     X_train_outer, X_val_outer = X_train.iloc[train_index], X_train.iloc[test_index]
#     y_train_outer, y_val_outer = y_train.iloc[train_index], y_train.iloc[test_index]

#     # Apply preprocessing (linear correction and Z-normalization) to X_train_outer and X_val_outer
#     # Linear correction
#     data_controls_train_outer = X_train_outer[y_train_outer == 0]  # Control group for the fold
#     for roi_feature in roi_features:
#         if roi_feature in X_train_outer.columns:
#             regr = LinearRegression()
#             regr.fit(data_controls_train_outer[['Sex', 'Age', 'DLICV_baseline']], data_controls_train_outer[roi_feature])
#             # Apply correction to the training set
#             correction_train = regr.predict(X_train_outer[['Sex', 'Age', 'DLICV_baseline']])
#             X_train_outer[roi_feature] -= correction_train

#             # Apply the same correction to the validation set
#             correction_val = regr.predict(X_val_outer[['Sex', 'Age', 'DLICV_baseline']])
#             X_val_outer[roi_feature] -= correction_val

#     # Z-normalization for training and validation sets
#     scaler = StandardScaler().fit(data_controls_train_outer[roi_features])
#     X_train_outer[roi_features] = scaler.transform(X_train_outer[roi_features])
#     X_val_outer[roi_features] = scaler.transform(X_val_outer[roi_features])

#     X_train_outer = X_train_outer.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)
#     X_val_outer = X_val_outer.drop(['Sex', 'Age', 'DLICV_baseline'], axis=1)

#     # Inner loop for hyperparameter tuning
#     inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
#     grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=inner_cv, scoring='accuracy', n_jobs=-1)
#     grid_search.fit(X_train_outer, y_train_outer)

#     # Train model with best parameters on the entire training set of the outer fold
#     best_model = grid_search.best_estimator_
#     best_model.fit(X_train_outer, y_train_outer)

#     # Evaluate model on the validation set of the outer fold
#     y_pred_val = best_model.predict(X_val_outer)
#     outer_fold_accuracy.append(best_model.score(X_val_outer, y_val_outer))
#     outer_fold_balanced_accuracy.append(balanced_accuracy_score(y_val_outer, y_pred_val))
#     outer_fold_precision.append(precision_score(y_val_outer, y_pred_val, average='weighted'))
#     outer_fold_recall.append(recall_score(y_val_outer, y_pred_val, average='weighted'))
#     outer_fold_f1.append(f1_score(y_val_outer, y_pred_val, average='weighted'))
#     outer_fold_auc.append(roc_auc_score(y_val_outer, best_model.predict_proba(X_val_outer), multi_class='ovr', average='weighted'))

#     # Evaluate model on the validation set of the outer fold
#     y_pred_val = best_model.predict(X_val_outer)
#     accuracy = best_model.score(X_val_outer, y_val_outer)
#     balanced_accuracy = balanced_accuracy_score(y_val_outer, y_pred_val)
#     precision = precision_score(y_val_outer, y_pred_val, average='weighted')
#     recall = recall_score(y_val_outer, y_pred_val, average='weighted')
#     f1 = f1_score(y_val_outer, y_pred_val, average='weighted')
#     auc = roc_auc_score(y_val_outer, best_model.predict_proba(X_val_outer), multi_class='ovr', average='weighted')

#     # Append metrics to lists
#     outer_fold_accuracy.append(accuracy)
#     outer_fold_balanced_accuracy.append(balanced_accuracy)
#     outer_fold_precision.append(precision)
#     outer_fold_recall.append(recall)
#     outer_fold_f1.append(f1)
#     outer_fold_auc.append(auc)


# # Calculate average metrics across all outer folds
# print("Average Metrics Across All Outer Folds:")
# print(f"Accuracy: {np.mean(outer_fold_accuracy)}")
# print(f"Balanced Accuracy: {np.mean(outer_fold_balanced_accuracy)}")
# print(f"Precision: {np.mean(outer_fold_precision)}")
# print(f"Recall: {np.mean(outer_fold_recall)}")
# print(f"F1 Score: {np.mean(outer_fold_f1)}")
# print(f"AUC: {np.mean(outer_fold_auc)}")
